In [1]:
import os
import sys
import pandas as pd
import numpy as np

from datasets import load_dataset, load_from_disk
import nltk 
nltk.download('punkt')

import evaluate

[nltk_data] Downloading package punkt to
[nltk_data]     /home/RDC/zinovyee.hub/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
dataset_curated = load_from_disk('../data/raw/conala')

In [3]:
train_df = pd.DataFrame(data={'question_id': dataset_curated['train']['question_id'], 
                            'intent' : dataset_curated['train']['intent'],
                            'rewritten_intent' : dataset_curated['train']['rewritten_intent'],
                            'snippet' : dataset_curated['train']['snippet']})

test_df = pd.DataFrame(data={'question_id': dataset_curated['test']['question_id'], 
                            'intent' : dataset_curated['test']['intent'],
                            'rewritten_intent' : dataset_curated['test']['rewritten_intent'],
                            'snippet' : dataset_curated['test']['snippet']})

full_df = pd.concat([train_df, test_df], axis=0)

In [4]:
full_df = full_df.sort_values("question_id").reset_index(drop=True)

In [5]:
print(full_df.question_id.nunique())

2074


In [6]:
qids = full_df.question_id.unique()
qids.sort()
print(qids)

[    1476     1854     3061 ... 42731970 42747987 42765620]


In [37]:
(2074-900)/35

33.542857142857144

In [8]:
i = 15 
(i+1)*900

14400

In [39]:
first_train_ids = qids[:900]
batches = []
batch_size = 35

for i in range(34):
    print(i)

    batch_start = 900+(i)*batch_size 
    print(batch_start)
    
    if i!=33:
        batch_end = batch_start + batch_size
        batches.append(qids[batch_start:batch_end])
        print(batch_end)
    else: 
        batches.append(qids[batch_start:])
        print(len(qids)-1)

0
900
935
1
935
970
2
970
1005
3
1005
1040
4
1040
1075
5
1075
1110
6
1110
1145
7
1145
1180
8
1180
1215
9
1215
1250
10
1250
1285
11
1285
1320
12
1320
1355
13
1355
1390
14
1390
1425
15
1425
1460
16
1460
1495
17
1495
1530
18
1530
1565
19
1565
1600
20
1600
1635
21
1635
1670
22
1670
1705
23
1705
1740
24
1740
1775
25
1775
1810
26
1810
1845
27
1845
1880
28
1880
1915
29
1915
1950
30
1950
1985
31
1985
2020
32
2020
2055
33
2055
2073


In [40]:
full_df['t_batch'] = -1
for i, batch_ids in enumerate(batches):
    full_df.loc[full_df.question_id.isin(set(batch_ids)), 't_batch'] = i

In [41]:
def postprocess_text(preds, labels):

    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    # rougeLSum expects newline after each sentence
    preds  = ["\n".join(nltk.sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(nltk.sent_tokenize(label)) for label in labels]
    return preds, labels

In [42]:
def batch_tokenize_preprocess(batch, tokenizer, max_input_length, max_output_length):

    source = batch["input_sequence"]
    target = batch["output_sequence"]

    source_tokenized = tokenizer(
        source, padding="max_length",
        truncation=True, max_length=max_input_length
    )

    target_tokenized = tokenizer(
        target, padding="max_length",
        truncation=True, max_length=max_output_length
    )

    batch = {k: v for k, v in source_tokenized.items()}

    # Ignore padding in the loss

    batch["labels"] = [
        [-100 if token == tokenizer.pad_token_id else token for token in l]
        for l in target_tokenized["input_ids"]
    ]

    return batch

In [43]:
def compute_metric_with_params(tokenizer, metrics_list=['rouge', 'bleu']):
    def compute_metrics(eval_preds):
    
        preds, labels = eval_preds
    
        if isinstance(preds, tuple):
            preds = preds[0]
    
        decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    
        # Replace -100 in the labels as we can't decode them.
        labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
        decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
        # POST PROCESSING
        decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)
    
        results_dict = {}
        for m in metrics_list:
            metric = evaluate.load(m)
    
            if m=='bleu':
                result = metric.compute(
                    predictions=decoded_preds, references=decoded_labels
                )
            elif m=='rouge':
                result = metric.compute(
                    predictions=decoded_preds, references=decoded_labels, use_stemmer=True
                )
            result = {key: value for key, value in result.items() if key!='precisions'}
    
            prediction_lens = [
                np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds
            ]
            result["gen_len"] = np.mean(prediction_lens)
            result = {k: round(v, 4) for k, v in result.items()}
            results_dict.update(result)
        return results_dict
    return compute_metrics

In [44]:
def generate_summary(test_samples, model, tokenizer, encoder_max_length, decoder_max_length):
    inputs = tokenizer(
        test_samples["input_sequence"],
        padding="max_length",
        truncation=True,
        max_length=encoder_max_length,
        return_tensors="pt",
    )
    input_ids = inputs.input_ids.to(model.device)
    attention_mask = inputs.attention_mask.to(model.device)
    outputs = model.generate(input_ids, attention_mask=attention_mask, max_new_tokens=decoder_max_length)
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    return outputs, output_str

In [45]:
def train_t5(): 
    pass

In [46]:
import torch

In [47]:
# CREATE ANALYSIS FOLDER
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

model_name="Salesforce/codet5-base-multi-sum"

cuda


In [48]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq, Seq2SeqTrainer

In [49]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name, skip_special_tokens=False)

In [50]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name, skip_special_tokens=False)

if True:
    for parameter in model.parameters():
        parameter.requires_grad = False
    for i, m in enumerate(model.decoder.block):        
        #Only un-freeze the last n transformer blocks in the decoder
        if i+1 > 12 - 4:
            for parameter in m.parameters():
                parameter.requires_grad = True

In [51]:
DECODER_LENGTH = 20
ENCODER_LENGTH = 15

In [52]:
model.to(device)
print(device)

cuda


In [53]:
from datasets import Dataset

In [54]:
def prep_for_hf(df: pd.DataFrame, batch_id: int): 
    df = df.rename(columns={'snippet': 'input_sequence', 
                    'intent' : 'output_sequence'})
    df = df.loc[df.t_batch==batch_id, ['input_sequence', 'output_sequence']]
    df = df.sample(frac=1, random_state=42)
    return Dataset.from_pandas(df)

In [55]:
train_dataset = prep_for_hf(full_df, -1)
test_dataset = prep_for_hf(full_df, 0)


In [57]:
train_data_txt = train_dataset
    
validation_data_txt = test_dataset

train_data = train_data_txt.map(
    lambda batch: batch_tokenize_preprocess(
        batch, 
        tokenizer=tokenizer,
        max_input_length=ENCODER_LENGTH,
        max_output_length=DECODER_LENGTH
    ),
    batch_size=8,
    batched=True,
    remove_columns=train_data_txt.column_names,
)

validation_data = validation_data_txt.map(
    lambda batch: batch_tokenize_preprocess(
        batch, 
        tokenizer=tokenizer,
        max_input_length=ENCODER_LENGTH,
        max_output_length=DECODER_LENGTH
    ),
    batched=True,
    remove_columns=validation_data_txt.column_names,
)


# SUBSAMPLE FOR GENERATION BEFORE TUNING
test_samples = validation_data_txt.select(range(20))
summaries_before_tuning = generate_summary(test_samples, 
                                            model, 
                                            tokenizer, 
                                            ENCODER_LENGTH,
                                            DECODER_LENGTH)[1]

Map:   0%|          | 0/1427 [00:00<?, ? examples/s]

Map:   0%|          | 0/42 [00:00<?, ? examples/s]

In [58]:
eval_columns_list = [
                                "eval_loss",
                                "eval_rouge1",
                                "eval_rouge2",
                                "eval_rougeL",
                                "eval_rougeLsum",
                                "eval_bleu",
                                "eval_gen_len",
                            ]

In [59]:
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["WANDB_DISABLED"] = "true"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [63]:
import gc
gc.collect()

torch.cuda.empty_cache()
training_args = Seq2SeqTrainingArguments(
    output_dir=f"rep/results",
    num_train_epochs=2,
    do_train=True,
    do_eval=True,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    learning_rate=5e-4,
    warmup_steps=100,
    weight_decay=0.1,
    label_smoothing_factor=0.1,
    predict_with_generate=True,
    logging_dir=f"rep/logs",
    logging_steps=100,
    save_total_limit=1,
    report_to=None,
    save_strategy='epoch',
    logging_strategy='epoch',
    evaluation_strategy='epoch',
    load_best_model_at_end=False    
)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

compute_metrics = compute_metric_with_params(tokenizer)

trainer = Seq2SeqTrainer(
model=model,
args=training_args,
data_collator=data_collator,
train_dataset=train_data,
eval_dataset=validation_data,
tokenizer=tokenizer,
compute_metrics=compute_metrics,
)

# ZERO - SHOT
results_zero_shot = trainer.evaluate()
results_zero_shot_df = pd.DataFrame(data=results_zero_shot, index=[0])[eval_columns_list]
results_zero_shot_df.loc[0, :] = results_zero_shot_df.loc[0, :].apply(lambda x: round(x, 3))
print(results_zero_shot_df)


# TRAINING
trainer.train()

# FINE-TUNING
results_fine_tune = trainer.evaluate()
results_fine_tune_df = pd.DataFrame(data=results_fine_tune, index=[0])[eval_columns_list]
results_fine_tune_df.loc[0, :] = results_fine_tune_df.loc[0, :].apply(lambda x: round(x, 3))
print(results_fine_tune_df)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


   eval_loss  eval_rouge1  eval_rouge2  eval_rougeL  eval_rougeLsum  \
0      5.535        0.161        0.019        0.133           0.137   

   eval_bleu  eval_gen_len  
0        0.0         9.688  


/home/RDC/zinovyee.hub/.local/lib/python3.9/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len,Bleu,Brevity Penalty,Length Ratio,Translation Length,Reference Length
1,3.873600,4.132750,0.336800,0.121200,0.279600,0.280200,12.604200,0.044600,1.000000,1.002200,462,461
2,2.937000,4.130136,0.284300,0.083300,0.245100,0.244300,11.666700,0.025700,0.939700,0.941400,434,461


/home/RDC/zinovyee.hub/.conda/envs/encode_code/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/RDC/zinovyee.hub/.conda/envs/encode_code/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/RDC/zinovyee.hub/.conda/envs/encode_code/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


   eval_loss  eval_rouge1  eval_rouge2  eval_rougeL  eval_rougeLsum  \
0       4.13        0.284        0.083        0.245           0.244   

   eval_bleu  eval_gen_len  
0      0.026        11.667  


In [64]:
summaries_after_tuning = generate_summary(test_samples, 
                                                model,
                                                tokenizer,
                                                ENCODER_LENGTH,
                                                DECODER_LENGTH)[1]
    
for i, description in enumerate(test_samples["output_sequence"]):
        print('_'*10)
        print(f'Original: {description}')
        
        print('\n')
        print(f'Summary before Tuning: {summaries_before_tuning[i]}')
        print('\n')
        print(f'Summary after Tuning: {summaries_after_tuning[i]}')
        print('\n')
        print('_'*10)
        print('\n'*2)

__________
Original: Removing duplicate characters from a string


Summary before Tuning: Join the set with foo.


Summary after Tuning: Python: How to convert a string to a string?


__________



__________
Original: finding index of an item closest to the value in a list that's not entirely sorted


Summary before Tuning: Returns the minimum element in an array.


Summary after Tuning: How to get the largest item in a list?


__________



__________
Original: (Django) how to get month name?


Summary before Tuning: Return today s string representation


Summary after Tuning: get current time


__________



__________
Original: sorting values of python dict using sorted builtin function


Summary before Tuning: Returns a sorted list of the keys in the dictionary.


Summary after Tuning: iterate over a dictionary in sorted order


__________



__________
Original: Convert an IP string to a number and vice versa


Summary before Tuning: Returns a string with the IPv6 address.


Summ

In [ ]:
loss

In [65]:
loss = []
rouge_1 = []
for i in range(1, 34): 
    print(i)
    test_dataset = prep_for_hf(full_df, i)
    validation_data_txt = test_dataset
    validation_data = validation_data_txt.map(
        lambda batch: batch_tokenize_preprocess(
            batch, 
            tokenizer=tokenizer,
            max_input_length=ENCODER_LENGTH,
            max_output_length=DECODER_LENGTH
        ),
        batched=True,
        remove_columns=validation_data_txt.column_names,
    )


    trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_data,
    eval_dataset=validation_data,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    )

    # ZERO - SHOT
    results_zero_shot = trainer.evaluate()
    results_zero_shot_df = pd.DataFrame(data=results_zero_shot, index=[0])[eval_columns_list]
    results_zero_shot_df.loc[0, :] = results_zero_shot_df.loc[0, :].apply(lambda x: round(x, 3))
    print(results_zero_shot_df)
    rouge_1.append(results_zero_shot_df['eval_rouge1'])
    loss.append(results_zero_shot_df['eval_loss'])


1


Map:   0%|          | 0/53 [00:00<?, ? examples/s]

/home/RDC/zinovyee.hub/.conda/envs/encode_code/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


   eval_loss  eval_rouge1  eval_rouge2  eval_rougeL  eval_rougeLsum  \
0      3.546        0.411        0.178        0.373           0.373   

   eval_bleu  eval_gen_len  
0      0.066        10.698  
2


Map:   0%|          | 0/40 [00:00<?, ? examples/s]

...

/home/RDC/zinovyee.hub/.conda/envs/encode_code/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


   eval_loss  eval_rouge1  eval_rouge2  eval_rougeL  eval_rougeLsum  \
0      4.162        0.277        0.074        0.247           0.248   

   eval_bleu  eval_gen_len  
0       0.03        12.071  
33


Map:   0%|          | 0/19 [00:00<?, ? examples/s]

/home/RDC/zinovyee.hub/.conda/envs/encode_code/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


   eval_loss  eval_rouge1  eval_rouge2  eval_rougeL  eval_rougeLsum  \
0       4.25        0.367        0.096         0.31           0.316   

   eval_bleu  eval_gen_len  
0      0.037        11.632  


In [ ]:
print(1)

In [ ]:
rouge_1